Descarga datos de rayos X blandos de GOES para n días seleccionados de forma semi-aleatoria y espaciados uniformemente.

Organiza los días en carpetas por bloques de hasta chunk_size días.

Descarga en las mismas carpetas los catálogos de flares correspondientes a esas fechas.

Sustrae el background aplicando el método de diferencias móviles.

Calcula temperatura y medida de emisión.

Normaliza la medida de emisión.

Consolida todos los bloques en un único archivo con los resultados completos.

# Función completa 

Calcula hasta tiempos de anticipación del peak cambiando solo fecha y grafica los tiempos que se quieran 

## Paquetes

In [67]:
from sunpy.net import Fido, attrs as a
from sunpy.timeseries import TimeSeries
from sunpy.timeseries.sources.goes import XRSTimeSeries
import astropy.units as u
from sunkit_instruments.goes_xrs import calculate_temperature_em
import matplotlib.pyplot as plt
import numpy as np
from sunpy.data import manager
import netCDF4 as nc
import os
import pandas as pd
from datetime import datetime, timedelta
import copy
from matplotlib import colormaps
list(colormaps)
#from colorspacious import cspace_converter
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
from matplotlib.ticker import LogFormatter
from matplotlib.ticker import LogFormatterMathtext
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import re
import glob
import logging

## Functions

In [68]:
##############################
##    1. Download GOES data   #
##############################

## 1.0. Ensure 1 min resolution (C1)
def ensure_1min_resolution(ts):
    """
    Revisa si el TimeSeries está en resolución de 1 minuto.
    Si no, lo re-muestrea a 1 min con la media.

    Check if the TimeSeries has a resolution of 1 minute.
    If no, it will be shown again in 1 minute with the media
    """
    # Pasar a DataFrame
    df = ts.to_dataframe()
    
    # Calcular la resolución actual (diferencia entre los 2 primeros tiempos)
    current_res = (df.index[1] - df.index[0]).total_seconds()
    
    if abs(current_res - 60) < 1:  # ya es 1 min (tolerancia de 1s)
        print("Resolution = 1 minute")
        return ts
    else:
        print(f"Resolution detected: {current_res:.2f} s → resampling at 1 min")
        df_resampled = df.resample("1min").mean()
        return TimeSeries(df_resampled, ts.meta)

## 1.1. Download data (C1)
def Download_Data(start_time, end_time, resolution="avg1m", log_file="errores_goes.log"):
    
    """
    Descarga datos GOES entre start_time y end_time, asegura resolución 1min,
    guarda gráficas en block_dir/data_graphs y log de errores en block_dir.

    Entrada / Input:
        start_time (str): Tiempo inicial del intervalo de búsqueda, en formato compatible con SunPy.
        end_time (str): Tiempo final del intervalo de búsqueda, en formato compatible con SunPy.
        resolution (str, opcional / optional): Resolución temporal de los datos GOES. 
            Opciones válidas: "flx1s", "avg1m". Por defecto es 'avg1m'.

    Salida / Output:
        TimeSeries: Objeto de SunPy que contiene los datos XRS del satélite GOES 
                    dentro del intervalo de tiempo especificado.
                    con la resolución especificada.

    Descripción / Description:
        Esta función busca, descarga y carga datos del instrumento GOES (X-Ray Sensor, XRS)
        en el intervalo de tiempo especificado y con la resolución deseada. Utiliza Fido para la búsqueda
        y retorna un objeto TimeSeries con los datos.

        This function searches for, downloads, and loads data from the GOES (X-Ray Sensor, XRS)
        within the specified time interval and chosen resolution. It uses Fido for querying and 
        returns a TimeSeries object with the data.
    """
    try:
        # ===== Directorios =====
        if block_dir is not None:
            os.makedirs(block_dir, exist_ok=True)
            graph_dir = os.path.join(block_dir, "data_graphs")
            os.makedirs(graph_dir, exist_ok=True)
            log_file = os.path.join(block_dir, "errores_goes.log")
        else:
            graph_dir = "data_graphs"
            os.makedirs(graph_dir, exist_ok=True)
            log_file = "errores_goes.log"

        # ===== Validar resolución =====
        valid_resolutions = ["flx1s", "avg1m"]
        if resolution not in valid_resolutions:
            raise ValueError(f"Resolución no válida. Usa una de: {valid_resolutions}")

        # ===== Buscar y descargar datos =====
        print(f"Buscando datos de: {start_time}")
        result = Fido.search(a.Time(start_time, end_time), a.Instrument.goes, a.Resolution(resolution))

        if len(result[0]) == 0:
            msg = f"No hay datos GOES para {start_time} - {end_time}. Día saltado."
            print(msg)
            with open(log_file, "a") as f:
                f.write(msg + "\n")
            return None, None  

        print(f"Descargando datos de {start_time}...")
        files = Fido.fetch(result)

        if len(files) == 0:
            msg = f"No se descargaron archivos GOES para {start_time}. Día saltado."
            print(msg)
            with open(log_file, "a") as f:
                f.write(msg + "\n")
            return None, None

        # ===== Cargar datos en TimeSeries =====
        try:
            ts = TimeSeries(files[0], source="XRS")
        except Exception as e:
            msg = f"Error al abrir archivo GOES de {start_time}: {e}"
            print(msg)
            with open(log_file, "a") as f:
                f.write(msg + "\n")
            return None, None

        # ===== Asegurar resolución 1 min =====
        goes_ts = ensure_1min_resolution(ts)

        # ===== Guardar gráfica =====
        fig, ax = plt.subplots(figsize=(12,6))
        goes_ts.plot(axes=ax)
        safe_time = start_time.replace(':','-').replace(' ','_')
        output_file = os.path.join(graph_dir, f"GOES_{safe_time}.png")
        fig.savefig(output_file, dpi=150, bbox_inches="tight")
        plt.close(fig)
        print(f"Gráfica guardada en {output_file}")

        # ===== Extraer observatorio =====
        try:
            meta0 = goes_ts.meta.metas[0]
            platform = meta0.get("platform", "g16")
            numero = int("".join(filter(str.isdigit, platform)))
            observatory = f"GOES-{numero}"
        except Exception:
            observatory = None
        print(f"Observatorio encontrado: {observatory}")

        return goes_ts, observatory

    except Exception as e:
        msg = f"Error inesperado al descargar datos GOES de {start_time}: {e}"
        print(msg)
        if block_dir is not None:
            with open(log_file, "a") as f:
                f.write(msg + "\n")
        return None, None  #  siempre devuelve una tupla



## 1.2. Truncar datos
def Truncate_Data(goes_ts, flare_start_time, flare_end_time):
    
    """
    Entrada / Input:
        goes_ts (TimeSeries): Objeto TimeSeries con los datos GOES completos.
        flare_start_time (str): Tiempo de inicio de la fulguración (en formato compatible con SunPy).
        flare_end_time (str): Tiempo de fin de la fulguración (en formato compatible con SunPy).

    Salida / Output:
        TimeSeries: Objeto TimeSeries con los datos recortados al intervalo de la fulguración.

    Descripción / Description:
        Esta función recorta un conjunto de datos GOES a un intervalo de tiempo específico
        correspondiente al inicio y fin de una fulguración solar. Si el intervalo no contiene datos,
        se lanza una excepción.

        This function trims a GOES TimeSeries dataset to a specific time interval
        corresponding to the start and end of a solar flare. If the interval contains no data,
        an exception is raised.
    """

    # Seleccionar el rango de interés / Select the time range of interest
    goes_flare = goes_ts.truncate(flare_start_time, flare_end_time)

    # Verificar si hay datos disponibles  / Check if data is available
    if len(goes_flare.to_dataframe()) == 0:
        raise ValueError("El rango de datos seleccionado está vacío. Revisa las fechas.")
        # The selected time range is empty. Please check the input times.

    # Visualizar los datos truncados / Plot the trimmed data
    goes_flare.peek()

    return goes_flare

## 2. Background (1)
def running_difference(goes_ts, Dif_time=5, plot=False, block_dir=None, start_time=None):
    """
    Calcula las diferencias de flujo de rayos X GOES a un intervalo definido (default 5 min).
    
    Parámetros
    ----------
    goes_ts : XRSTimeSeries
        Serie temporal original de GOES.
    Dif_time : int, optional
        Intervalo de diferencia en número de pasos para restar flux (default=5).
    plot : bool, optional
        Si True, guarda gráficas comparativas original vs corregido.
    block_dir : str, optional
        Carpeta base donde guardar gráficas.
    start_time : str, optional
        Tiempo inicial usado para nombrar archivos.
        
    Retorna
    -------
    goes_diff_ts : XRSTimeSeries
        Serie temporal corregida con las diferencias.
    """

    import matplotlib.dates as mdates
    from matplotlib.ticker import LogFormatterMathtext

    # --- 1. Extraer datos ---
    df = goes_ts.to_dataframe()
    flux_xrsa = df["xrsa"]
    flux_xrsb = df["xrsb"]
    npts = len(df)

    # --- 2. Calcular diferencias ---
    diffa = np.array(flux_xrsa[Dif_time:]) - np.array(flux_xrsa[:npts - Dif_time])
    diffb = np.array(flux_xrsb[Dif_time:]) - np.array(flux_xrsb[:npts - Dif_time])

    # --- 3. Llenar arreglos completos ---
    diffa_full = np.zeros(npts)
    diffb_full = np.zeros(npts)
    diffa_full[Dif_time:] = diffa
    diffb_full[Dif_time:] = diffb

    # --- 4. Crear DataFrame corregido ---
    df_diff = pd.DataFrame({'xrsa': diffa_full, 'xrsb': diffb_full}, index=df.index)

    # --- 5. Crear TimeSeries corregida ---
    units = {'xrsa': u.W / u.m**2, 'xrsb': u.W / u.m**2}
    goes_diff_ts = XRSTimeSeries(df_diff, units=units, meta=goes_ts.meta)

    # --- 6. Función auxiliar para graficar ---
    def save_plot(df_orig, df_corr, output_file, title="", logscale=False, positive_only=False):
        df_o, df_c = df_orig.copy(), df_corr.copy()
        if positive_only:
            df_o = df_o.clip(lower=1e-9)
            df_c = df_c.clip(lower=1e-9)

        fig, ax = plt.subplots(figsize=(12,6))
        ax.plot(df_o.index, df_o['xrsa'], label='XRSA (original)', color='blue')
        ax.plot(df_c.index, df_c['xrsa'], label='XRSA (corrected)', color='blue', linestyle='--')
        ax.plot(df_o.index, df_o['xrsb'], label='XRSB (original)', color='red')
        ax.plot(df_c.index, df_c['xrsb'], label='XRSB (corrected)', color='red', linestyle='--')

        date_only = df_orig.index[0].strftime("%Y-%m-%d")
        ax.set_xlabel(f"Time (UTC) — {date_only}")
        ax.set_ylabel("Flux [W/m²]")
        ax.set_title(title)
        ax.legend()
        ax.grid(True, which='both', ls='--', alpha=0.6)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

        if logscale:
            ax.set_yscale('log', base=10)
            ax.yaxis.set_major_formatter(LogFormatterMathtext())

        plt.tight_layout()
        fig.savefig(output_file, dpi=150, bbox_inches="tight")
        plt.close(fig)
        print(f"Plot saved at: {output_file}")

    # --- 7. Guardar gráficas si plot=True ---
    if plot and block_dir is not None and start_time is not None:
        graph_dir = os.path.join(block_dir, "data_graphs")
        os.makedirs(graph_dir, exist_ok=True)

        safe_time = start_time.replace(':','-').replace(' ','_')
        df_corr = df_diff

        # Linear
        save_plot(df, df_corr, os.path.join(graph_dir, f"GOES_diff_linear_{safe_time}.png"),
                  title=f"GOES Data Comparison (Δt={Dif_time} steps)")

        # Logarithmic
        save_plot(df, df_corr, os.path.join(graph_dir, f"GOES_diff_log_{safe_time}.png"),
                  title=f"GOES Data Comparison (Δt={Dif_time} steps) [Log Y]",
                  logscale=True)

        # Positive only
        save_plot(df, df_corr, os.path.join(graph_dir, f"GOES_diff_positive_{safe_time}.png"),
                  title=f"GOES Data Comparison [Positive only, Log Y]",
                  logscale=True,
                  positive_only=True)

    return goes_diff_ts


#####################################
# 5. Temperatura y Emission Measure #
#####################################

## 5.1. Calcular la Temperatura y Medida de emisión con los datos corregidos (C1)
#Calculate_Tem_EM(goes_flare_corrected, abundance='photospheric')
#Calculate_Tem_EM(goes_flare_corrected, abundance='coronal')
def Calculate_Tem_EM(goes_flare_corrected, abundance='coronal'):
    
    """
    Entrada / Input:
        goes_flare_corrected: Objeto TimeSeries corregido que contiene los datos del flare solar en los
        canales XRSA y XRSB. Debe ser compatible con la función `calculate_temperature_em` de SunPy.
        abundance (str): Tipo de abundancia elemental a usar en el cálculo (por defecto: 'coronal').
                         Otras opciones posibles incluyen 'photospheric', dependiendo del modelo de SunPy.

    Salida / Output:
        temp_em: Objeto que contiene la temperatura (T) y medida de emisión (EM) derivadas a partir de los
        datos GOES corregidos.

    Descripción / Description:
        Esta función utiliza los datos corregidos del satélite GOES para calcular la temperatura del plasma
        y la medida de emisión (EM) durante un evento de fulguración solar. Permite especificar el modelo
        de abundancia elemental a utilizar en el cálculo.

        This function uses corrected GOES data to compute the plasma temperature and emission measure (EM)
        during a solar flare. It allows specifying the elemental abundance model to be used in the calculation.

    Notas / Notes:
        - Usa la función `calculate_temperature_em` de SunPy.
        - El parámetro `abundance` controla el modelo de abundancias (por ejemplo, 'coronal' o 'photospheric').
        - Se desactiva temporalmente la verificación del hash de calibración del instrumento.
        - Los datos deben estar previamente corregidos y limpios.
    """

    print(f'Ahora vamos a calcular la T y EM con el modelo de abundancias:{abundance}')
    #  Saltar la verificación del hash temporalmente
    with manager.skip_hash_check():
        #temp_em = calculate_temperature_em(goes_flare_corrected, abundance='coronal')
        temp_em = calculate_temperature_em(goes_flare_corrected, abundance)
    
    print(f'se calculó T y EM con el modelo de abundancias:{abundance}')
    #print(temp_em)
    return temp_em


# mostrar un rango de datos de un TimeSeries
def show_range_ts(ts, start_idx, end_idx, height=300):
    df = ts.to_dataframe()
    subdf = df.iloc[start_idx:end_idx]
    display(HTML(f"""
    <div style="height:{height}px; overflow:auto; border:1px solid #ccc; padding:10px">
        {subdf.to_html()}
    </div>
    """))

# Uso:
#show_range_ts(goes_ts01, 0, 10)

#####################################
#     6. Calcula tiempos de FAI     #
#####################################

def calcular_fai_times(df, 
                       T_min=7, T_max=14, 
                       EM_threshold=0.005, 
                       col_T="T_cor", 
                       col_EM="EM_cor_norm"):
    """
    Calcula los tiempos en los que se cumplen los criterios del índice FAI.
    """

    # Copiar para no modificar el original
    df_fai = df.copy()

    # Condiciones del criterio FAI
    fai_condition = (
        (df_fai[col_T] >= T_min) & 
        (df_fai[col_T] <= T_max) & 
        (df_fai[col_EM] > EM_threshold)
    )

    # Selección
    df_fai_selected = df_fai[fai_condition]
    fai_times = df_fai_selected.index

    # Mostrar los resultados como en tu código original
    print(f"Se encontraron {len(df_fai_selected)} puntos que cumplen el criterio FAI.\n")
    print(df_fai_selected[[col_T, col_EM]].head())

    return fai_times, df_fai_selected

########################
# 7. Descargar Flares #
#######################

# Configurar logging básico
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

## 7.1. Descarga de datos de flares
def get_flares(start_time, end_time):
    """
    Busca fulguraciones solares reportadas por GOES en el intervalo dado.
    
    Parameters
    ----------
    start_time : str
        Tiempo inicial (YYYY-MM-DD o compatible con SunPy).
    end_time : str
        Tiempo final (YYYY-MM-DD o compatible con SunPy).
    
    Returns
    -------
    pd.DataFrame or None
        DataFrame con columnas ['StartTime', 'EndTime', 'Class', 'Observatory', 'PeakTime'] 
        o None si no se encontraron flares.
    """
    try:
        logging.info(f"Searching for GOES flares between {start_time} and {end_time}...")
        result = Fido.search(a.Time(start_time, end_time), a.hek.FL, a.hek.OBS.Observatory == "GOES")

        if not result or len(result) == 0 or len(result[0]) == 0:
            logging.info(f"No solar flares found between {start_time} and {end_time}.")
            return None

        # Filtrar columnas 1D
        names = [name for name in result[0].colnames if len(result[0][name].shape) <= 1]
        table = result[0][names].to_pandas()

        # Seleccionar columnas de interés
        flare_data = table[[
            "event_starttime",
            "event_endtime",
            "fl_goescls",
            "obs_observatory",
            "event_peaktime"
        ]]

        # Renombrar columnas
        flare_data.columns = ["StartTime", "EndTime", "Class", "Observatory", "PeakTime"]

        logging.info(f"Found {len(flare_data)} GOES solar flares between {start_time} and {end_time}.")
        return flare_data

    except Exception as e:
        logging.error(f"Error retrieving flares between {start_time} and {end_time}: {e}")
        return None
    

def build_full_dataframe(goes_ts, goes_corrected, temp_em_cor, temp_em_phot,
                         clip_negative=True, normalize_em=False):
    """
    Combina datos originales, corregidos y parámetros de temperatura/EM
    en un solo DataFrame.

    Parámetros
    ----------
    goes_ts : sunpy.timeseries.TimeSeries
        Serie temporal GOES remuestreada (contiene 'xrsa', 'xrsb').
    goes_corrected : sunpy.timeseries.TimeSeries
        Serie temporal con GOES corregido (xrsa, xrsb corregidos).
    temp_em_cor : sunpy.timeseries.TimeSeries
        Serie temporal con temperatura y EM coronal.
    temp_em_phot : sunpy.timeseries.TimeSeries
        Serie temporal con temperatura y EM fotosférica.
    clip_negative : bool, opcional
        Si True, reemplaza valores negativos con NaN (en lugar de 0).
    normalize_em : bool, opcional
        Si True, normaliza EM a unidades de 1e49 cm^-3.

    Retorna
    -------
    pd.DataFrame
        DataFrame combinado con todas las columnas.
    """

    # Originales
    df_original = goes_ts.to_dataframe()[['xrsa', 'xrsb']]

    # Corregidos
    df_corr = goes_corrected.to_dataframe().rename(
        columns={'xrsa': 'xrsa_corr', 'xrsb': 'xrsb_corr'}
    )

    # Coronal
    df_cor = temp_em_cor.to_dataframe()[['temperature', 'emission_measure']].rename(
        columns={'temperature': 'T_cor', 'emission_measure': 'EM_cor'}
    )

    # Fotosférica
    df_phot = temp_em_phot.to_dataframe()[['temperature', 'emission_measure']].rename(
        columns={'temperature': 'T_phot', 'emission_measure': 'EM_phot'}
    )

    # Combinar todo
    df_full = pd.concat([df_original, df_corr, df_cor, df_phot], axis=1)

    # Opcional: reemplazar valores negativos
    if clip_negative:
        df_full = df_full.mask(df_full < 0, np.nan)   # ahora quedan NaN y no 0

    # Opcional: normalizar EM
    if normalize_em:
        df_full['EM_cor_norm'] = df_full['EM_cor'] / 1e49
        df_full['EM_phot_norm'] = df_full['EM_phot'] / 1e49

    return df_full


# agrupa por clases de flares en 2 grupos por clase
def assign_flare_group(flare_class):
    """
    Asigna un grupo a un flare según su clase y subnivel.
    
    Ejemplos:
    - C3.0 → C1-4
    - C7.5 → C5-9
    - X2.0 → X1-4
    - X7.0 → X5+
    """
    group_ranges = {
        "A": [(1, 4), (5, 9)],
        "B": [(1, 4), (5, 9)],
        "C": [(1, 4), (5, 9)],
        "M": [(1, 4), (5, 9)],
        "X": [(1, 4), (5, 1000)]  # 1000 actúa como "infinito"
    }

    try:
        letter = flare_class[0]  # A, B, C, M, X
        number = float(flare_class[1:])  # acepta decimales, ej: "2.7" → 2.7
    except:
        return flare_class  # si algo raro viene en el CSV

    if letter not in group_ranges:
        return flare_class

    for (low, high) in group_ranges[letter]:
        if low <= number <= high:
            if high >= 1000:  # caso abierto (ej: X5+)
                return f"{letter}{low}+"
            return f"{letter}{low}-{high}"
    
    return flare_class


#######################
### cuenta los días ###
#######################

def count_days(start_date, end_date):
    """
    Counts the number of days between two dates.

    Parameters:
    - start_date (str): Start date in 'YYYY-MM-DD' format
    - end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
    - int: Number of days between start_date and end_date
    """
    # Convert the strings to datetime objects
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Calculate the difference
    difference = end - start
    
    return difference.days

# Example usage
#days = count_days("1980-01-05", "2025-08-20")
#print(f"Days between the dates: {days}")


#############################################
# elige los días cada cierto paso según     #
# el número de días que quiera analizar (n) #
#############################################

def select_dates(start_date, end_date, n=10):
    """
    Selects n dates evenly spaced between start_date and end_date.

    Parameters:
    - start_date (str): Start date in 'YYYY-MM-DD' format
    - end_date (str): End date in 'YYYY-MM-DD' format
    - n (int): Number of dates to select (default 10)

    Returns:
    - list of str: List of dates in 'YYYY-MM-DD' format
    """
    # Convert strings to datetime objects
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Calculate total number of days
    total_days = (end - start).days
    
    # Calculate integer step
    #step = total_days // (n - 1)  # n-1 intervals for n dates
    
    if n < 2:
        return [start_date]

    # step aproximado al entero más cercano
    step = max(1, int(round(total_days / (n - 1))))
    print(step)
    # Generate the dates
    dates = [start + timedelta(days=i*step) for i in range(n)]
    print(len(dates))
    # Convert to strings
    return [d.strftime("%Y-%m-%d") for d in dates]


# guarda los datos seleccionados segun el paso en un df
def dates_to_dataframe(dates_list):
    """
    Converts a list of dates into a DataFrame with start_time and end_time.

    Parameters:
    - dates_list (list of str): List of dates in 'YYYY-MM-DD' format

    Returns:
    - pd.DataFrame: DataFrame with columns ['start_time', 'end_time']
    """
    data = {
        "start_time": [f"{date} 00:00:00" for date in dates_list],
        "end_time":   [f"{date} 23:59:00" for date in dates_list]
    }
    return pd.DataFrame(data)


# Divide un dateframen en lotes:
def chunk_dataframe(df, chunk_size=20):
    """
    Divide un DataFrame en bloques (chunks) de tamaño chunk_size.
    Devuelve una lista de DataFrames.
    """
    return [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

# unir todos los bloques al final
def combine_blocks(output_dir, pattern, final_name, time_column=None):
    """
    Une todos los CSV de bloques en un único archivo final.
    Si los CSV tienen la primera columna como fecha sin nombre, se usa para ordenar.
    """
    files = sorted(glob.glob(os.path.join(output_dir, "**", f"*{pattern}*.csv"), recursive=True))
    df_list = []

    for f in files:
        # Leer CSV; si la primera columna no tiene nombre, le ponemos "time"
        df = pd.read_csv(f)
        if df.columns[0] == "":
            df = pd.read_csv(f, names=["time"] + list(df.columns[1:]), header=0)
        df_list.append(df)

    if df_list:
        combined = pd.concat(df_list, ignore_index=True)

        # Ordenar según columna de tiempo
        if time_column is None:
            # usar la primera columna como tiempo
            combined.iloc[:, 0] = pd.to_datetime(combined.iloc[:, 0])
            combined = combined.sort_values(by=combined.columns[0]).reset_index(drop=True)
            print(f"✅ Ordenado por la primera columna (usada como tiempo)")
        elif time_column in combined.columns:
            combined[time_column] = pd.to_datetime(combined[time_column])
            combined = combined.sort_values(by=time_column).reset_index(drop=True)
            print(f"✅ Ordenado por columna '{time_column}'")
        else:
            print(f"⚠️ Columna '{time_column}' no encontrada, no se ordenó")

        combined.to_csv(os.path.join(output_dir, final_name), index=False)
        print(f"{final_name} creado con {len(combined)} filas (desde {len(files)} archivos).")
    else:
        print(f"No se encontraron archivos con patrón {pattern}")

# Example usage
#dates_list = select_dates("1980-01-05", "2025-08-20", n=10)
#df_intervals = dates_to_dataframe(dates_list)

#print(df_intervals)



## Prueba función completa

In [69]:
def download_goes_flare_data(start_time, end_time, 
                      resolution="avg1m",  #resolución de descarga de los datos GOES (1min)
                      Dif_time=5,            #Difference time Δt (for background)
                      plot_diff=True):
    """
    Pipeline completo para análisis de datos GOES y cálculo de FAI y anticipación.

    Parámetros:
    -----------
    start_time : str
        Tiempo inicial (ej: "2017-09-06 12:00:00")
    end_time : str
        Tiempo final (ej: "2017-09-06 12:15:00")
    resolution : str, opcional
        Resolución de datos GOES, default "avg1m". Opciones: ["flx1s", "avg1m"]
    Dif_time : int, opcional
        Ventana en pasos para calcular diferencias (default 5).
    plot_diff : bool, opcional
        Si True, grafica las diferencias calculadas.
    
    Retorna:
    --------
    dict con:
        - df_full : DataFrame con todos los cálculos
        - df_flare_data : DataFrame de flares GOES en el intervalo
    """

    # 1. Descargar datos GOES
    print("1. Descargar datos GOES")
    data = Download_Data(start_time, end_time, resolution)
    # para días sin datos GOES
    if data is None:
        print(f"No hay datos GOES para {start_time} - {end_time}. Día saltado.\n")
        return None
    else:
        print(f"Se encontraron datos  GOES para {start_time} - {end_time}. Día saltado.\n")
        goes_ts, observatory = data
        # Convertir TimeSeries a DataFrame para inspección
        df_goes = goes_ts.to_dataframe()

        # Número de registros
        print(f"Número de registros: {len(df_goes)}")

        # Columnas disponibles
        print(f"Columnas disponibles: {list(df_goes.columns)}")
        
    print("2. Restar Background")
    # 2. Calcular diferencias
    goes_ts_corrected_diff = running_difference(goes_ts, Dif_time=Dif_time, plot=plot_diff, block_dir=block_dir, start_time=start_time)
                            

    # 3. Calcular T y EM (coronal y fotosférico)
    print("3. USAR FUNCIÓN SUNPY calculate_t_em")
    temp_em_cor = Calculate_Tem_EM(goes_ts_corrected_diff, abundance='coronal')
    temp_em_phot = Calculate_Tem_EM(goes_ts_corrected_diff, abundance='photospheric')

    print("4. CONSTRUIR df_full")
    # 4. Construir dataframe completo
    df_full = build_full_dataframe(goes_ts, goes_ts_corrected_diff, temp_em_cor, temp_em_phot,
                         clip_negative=True, normalize_em=True)

    print(f"5. añadiendo observatorio: GOES: {observatory}")
    # 5. Añadir columna del observatorio
    df_full["observatory"] = observatory
    # Reemplazar NaN por "Unknown"
    df_full["observatory"] = df_full["observatory"].fillna("Unknown")
    # Mover "observatory" al inicio
    cols = ["observatory"] + [col for col in df_full.columns if col != "observatory"]
    df_full = df_full[cols]
    #print(df_full)
    print(f"se añadió observatorio: GOES: {observatory}")

    #print(f"6. Normalizando EM")
    # 6. Normalizar EM
    #df_full['EM_cor_norm'] = df_full['EM_cor'] / 1e49
    #df_full['EM_phot_norm'] = df_full['EM_phot'] / 1e49
    
    print(f"6. Descargando flares: {start_time} - {end_time}")
    # 6. Descargar flares
    flare_data = get_flares(start_time, end_time)
    
    #verifica si encontró flares para el día
    if flare_data is None:
        print(f"No se encontraron flares para el intervalo {start_time} - {end_time}. Saltando...")
        # Crear DataFrames vacíos para todos los resultados que dependen de flares
        df_flare_data = pd.DataFrame()
    
        
        # Retornar diccionario con flares vacíos
        return {
            "df_full": df_full,
            "df_flare_data": df_flare_data,
        }
    

    # Si flare_data no es None continúa
    print(f"Se descargaron flares para el intervalo {start_time} - {end_time}.")
    df_flare_data = flare_data[
        flare_data['Class'].notna() & 
        (flare_data['Class'].str.strip() != "") &
        (flare_data['Observatory'] == "GOES")
    ]
    print(f"7. Se filtraron solo flares de GOES.")
      

    # Retornar todo como dict
    return {
        "df_full": df_full,
        "df_flare_data": df_flare_data,
    }

## Count days

## New folder for data

In [74]:
# ==========================
# CONFIGURACIÓN
# ==========================
date_str = "2008-06-10"   # 👉 cambia esta fecha al día que quieras procesar
resolution = "avg1m"
Dif_time = 5
plot_diff = True

# Construir los límites del día
start_time = f"{date_str} 00:00:00"
end_time   = f"{date_str} 23:59:59"

# Crear carpeta de salida
fecha_actual = datetime.now().strftime("%Y-%m-%d")
output_dir = f"{fecha_actual}_Single_Day_GOES"
os.makedirs(output_dir, exist_ok=True)
print(f"📁 Carpeta creada: {output_dir}")

# Crear subcarpeta para el día
block_dir = os.path.join(output_dir, f"Day_{date_str}")
os.makedirs(block_dir, exist_ok=True)

# Archivos esperados
file_full  = os.path.join(block_dir, f"df_full_{date_str}.csv")
file_flare = os.path.join(block_dir, f"df_flare_data_{date_str}.csv")

# Si ya existen ambos, saltamos este día
if os.path.exists(file_full) and os.path.exists(file_flare):
    print(f"⚠️ El día {date_str} ya fue procesado, saltando...")
else:
    print(f"🚀 Procesando datos para el día {date_str}...")
    
    try:
        results = download_goes_flare_data(start_time, end_time, 
                                           resolution=resolution, 
                                           Dif_time=Dif_time, 
                                           plot_diff=plot_diff)
    except Exception as e:
        print(f"❌ Error al procesar el día {date_str}: {e}")
        results = None

    if results is None:
        print(f"⚠️ No se obtuvieron resultados para {date_str}.")
    else:
        df_full = results["df_full"]
        df_flare_data = results["df_flare_data"]

        # Guardar si hay datos
        if not df_full.empty:
            df_full.to_csv(file_full, index=True)
            print(f"✅ df_full guardado: {file_full}")
        else:
            print(f"⚠️ df_full vacío, no se guardó archivo.")

        if not df_flare_data.empty:
            df_flare_data.to_csv(file_flare, index=True)
            print(f"✅ df_flare_data guardado: {file_flare}")
        else:
            print(f"⚠️ df_flare_data vacío, no se guardó archivo.")

    print(f"🏁 Día {date_str} procesado correctamente.")

📁 Carpeta creada: 2025-10-08_Single_Day_GOES
🚀 Procesando datos para el día 2008-06-10...
1. Descargar datos GOES
Buscando datos de: 2008-06-10 00:00:00


Descargando datos de 2008-06-10 00:00:00...


Files Downloaded: 100%|██████████| 1/1 [00:00<00:00,  1.75file/s]


Resolution = 1 minute
Gráfica guardada en 2025-10-08_Single_Day_GOES/Day_2008-06-10/data_graphs/GOES_2008-06-10_00-00-00.png
Observatorio encontrado: GOES-10
Se encontraron datos  GOES para 2008-06-10 00:00:00 - 2008-06-10 23:59:59. Día saltado.

Número de registros: 1440
Columnas disponibles: ['xrsa', 'xrsb', 'xrsa_quality', 'xrsb_quality']
2. Restar Background
Plot saved at: 2025-10-08_Single_Day_GOES/Day_2008-06-10/data_graphs/GOES_diff_linear_2008-06-10_00-00-00.png
Plot saved at: 2025-10-08_Single_Day_GOES/Day_2008-06-10/data_graphs/GOES_diff_log_2008-06-10_00-00-00.png


/tmp/ipykernel_1687418/4009928038.py:226: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_o = df_o.clip(lower=1e-9)


...

se calculó T y EM con el modelo de abundancias:coronal
Ahora vamos a calcular la T y EM con el modelo de abundancias:photospheric


Files Downloaded: 100%|██████████| 1/1 [00:01<00:00,  1.06s/file]
2025-10-08 16:30:27 - root - INFO: Searching for GOES flares between 2008-06-10 00:00:00 and 2008-06-10 23:59:59...


se calculó T y EM con el modelo de abundancias:photospheric
4. CONSTRUIR df_full
5. añadiendo observatorio: GOES: GOES-10
se añadió observatorio: GOES: GOES-10
6. Descargando flares: 2008-06-10 00:00:00 - 2008-06-10 23:59:59


2025-10-08 16:30:28 - root - INFO: No solar flares found between 2008-06-10 00:00:00 and 2008-06-10 23:59:59.


No se encontraron flares para el intervalo 2008-06-10 00:00:00 - 2008-06-10 23:59:59. Saltando...
✅ df_full guardado: 2025-10-08_Single_Day_GOES/Day_2008-06-10/df_full_2008-06-10.csv
⚠️ df_flare_data vacío, no se guardó archivo.
🏁 Día 2008-06-10 procesado correctamente.


In [71]:
# Unir cada salida
combine_blocks(output_dir, "df_full_block", f"all_df_full_{n}.csv", time_column=None)
combine_blocks(output_dir, "df_flare_data_block", f"all_df_flare_data_{n}.csv", time_column="StartTime")


No se encontraron archivos con patrón df_full_block
No se encontraron archivos con patrón df_flare_data_block
